<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Determine-nonviable-genes-of-iterative-MaxMass-KOs" data-toc-modified-id="Determine-nonviable-genes-of-iterative-MaxMass-KOs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Determine nonviable genes of iterative MaxMass KOs</a></span></li></ul></div>

# Determine nonviable genes of iterative MaxMass KOs


In [80]:
from cobra.flux_analysis import single_gene_deletion, double_gene_deletion
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [78]:
def new_nonviable_genes_of_ko( model, kos, previously_nonviable_genes=[], viability_tol = 0.9 ):
    """
    Arguments:
       model --  A cobra model
       kos -- A list of genes
       previously_essential_genes --  The list of genes already known to be essential
    Returns -- A list of new essential genes not in the list of previously essential genes
    """

    previous_growth = model.slim_optimize()
    for ko in kos:
        if model.genes.has_id( ko ):
            model.genes.get_by_id( ko ).knock_out()
    ko_growth = model.slim_optimize()
    if ko_growth > viability_tol*previous_growth:
        growth_of_single_gene_ko = clean_single_gene_deletion( model )
        nonviable_genes = growth_of_single_gene_ko[growth_of_single_gene_ko['growth'] 
                                                   <
                                                   viability_tol*ko_growth]
        new_nonviable_genes = nonviable_genes.loc[ set(previously_nonviable_genes)
                                                       - 
                                                       set(nonviable_genes.index) ]
        return new_nonviable_genes, model
    else:
        print("KO growth of {} is less than {}% of wild-type growth {}".format( ko_growth, 
                                                                                viability_tol*100, 
                                                                                previous_growth ))
    

def clean_single_gene_deletion( model ):
    sgd = single_gene_deletion( model )
    sgd.index = [list(gene)[0] for gene in sgd.index]
    return sgd

def nonviable_genes_of_maxmass( base_model, maxmass_solutions , viability_tol ):
    growth_of_base_single_gene_kos = clean_single_gene_deletion( base_model )
    base_growth = base_model.slim_optimize()
    base_nonviable_genes = growth_of_base_single_gene_kos[ growth_of_base_single_gene_kos['growth'] 
                                                           < 
                                                           base_growth*viability_tol ]
    previous_nonviable_genes = base_nonviable_genes.copy()
    previous_nonviable_genes['SolutionOrder'] = 'base'
    new_model = base_model.copy()
    for i in sorted(set(maxmass_solutions['SolutionOrder'].values),
                    reverse=True):
        maxmass_kos = maxmass_solutions[ maxmass_solutions['SolutionOrder'] 
                                         == 
                                         i ].index
        new_nonviable_genes, new_model = new_nonviable_genes_of_ko( 
                                                         new_model, 
                                                         maxmass_kos,
                                                         previous_nonviable_genes.index,
                                                         viability_tol )
        new_nonviable_genes['SolutionOrder'] = i
        previous_nonviable_genes = previous_nonviable_genes.append( new_nonviable_genes )
    return previous_nonviable_genes
        

In [79]:
w3110_wt = cobra.io.read_sbml_model('data/Ecoli/iJO1366.xml')

w3110_control_ko_solutions = pd.read_csv('out/W3110_control_genomeview.tab', 
                                         sep='\t',
                                         index_col='$gene_or_promoter')
viability_tol = 0.9
nonviable_genes = nonviable_genes_of_maxmass( w3110_wt, w3110_control_ko_solutions, viability_tol )
nonviable_genes

,growth,status,SolutionOrder
b2472,-0.00,optimal,base
b3607,-0.00,optimal,base
b3734,0.40,optimal,base
b2764,0.00,optimal,base
b3857,0.00,optimal,base
b0074,0.00,optimal,base
b2284,0.87,optimal,base
b2557,0.00,optimal,base
b3997,0.00,optimal,base
b4039,0.00,optimal,base


In [ ]:
double_ko = double_gene_deletion( w3110_wt ) 